# Data Cleaning

Here I am cleaning the datasets I chose to vizualize, analize and predict consommation and production.
Each datasets can be found [here](https://opendata.reseaux-energies.fr/explore/), their licences are [Etalab](https://www.etalab.gouv.fr/wp-content/uploads/2017/04/ETALAB-Licence-Ouverte-v2.0.pdf)

In [24]:
import pandas as pd
import numpy as np
import feather
import datetime

# Colonnes à renommer pour plus de facilités 
col_names = ['code_insee_region', 'region', 'nature', 'date', 'heure', 'date_heure', 'consommation', 'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'pompage', 'bioenergies', 'ech_physiques', 'flux_physiques_d_auvergne_rhone_alpes_vers_grand_est', 'flux_physiques_de_bourgogne_franche_comte_vers_grand_est', 'flux_physiques_de_bretagne_vers_grand_est', 'flux_physiques_de_centre_val_de_loire_vers_grand_est', 'flux_physiques_de_grand_est_vers_grand_est', 'flux_physiques_de_hauts_de_france_vers_grand_est', 'flux_physiques_d_ile_de_france_vers_grand_est', 'flux_physiques_de_normandie_vers_grand_est', 'flux_physiques_de_nouvelle_aquitaine_vers_grand_est', 'flux_physiques_d_occitanie_vers_grand_est', 'flux_physiques_de_pays_de_la_loire_vers_grand_est', 'flux_physiques_de_paca_vers_grand_est', 'flux_physiques_de_grand_est_vers_auvergne_rhone_alpes', 'flux_physiques_de_grand_est_vers_bourgogne_franche_comte', 'flux_physiques_de_grand_est_vers_bretagne', 'flux_physiques_de_grand_est_vers_centre_val_de_loire', 'flux_physiques_de_grand_est_vers_grand_est0', 'flux_physiques_de_grand_est_vers_hauts_de_france', 'flux_physiques_de_grand_est_vers_ile_de_france', 'flux_physiques_de_grand_est_vers_normandie', 'flux_physiques_de_grand_est_vers_nouvelle_aquitaine', 'flux_physiques_de_grand_est_vers_occitanie', 'flux_physiques_de_grand_est_vers_pays_de_la_loire', 'flux_physiques_de_grand_est_vers_paca', 'flux_physiques_allemagne_vers_grand_est', 'flux_physiques_belgique_vers_grand_est', 'flux_physiques_espagne_vers_grand_est', 'flux_physiques_italie_vers_grand_est', 'flux_physiques_luxembourg_vers_grand_est', 'flux_physiques_royaume_uni_vers_grand_est', 'flux_physiques_suisse_vers_grand_est', 'flux_physiques_de_grand_est_vers_allemagne', 'flux_physiques_de_grand_est_vers_belgique', 'flux_physiques_de_grand_est_vers_espagne' , 'flux_physiques_de_grand_est_vers_italie', 'flux_physiques_de_grand_est_vers_luxembourg', 'flux_physiques_de_grand_est_vers_royaume_uni', 'flux_physiques_de_grand_est_vers_suisse', 'tco_thermique', 'tch_thermique', 'tco_nucleaire', 'tch_nucleaire', 'tco_eolien', 'tch_eolien', 'tco_solaire', 'tch_solaire', 'tco_hydraulique', 'tch_hydraulique', 'tco_bioenergies', 'tch_bioenergies']
# na_values observées dans le jeu de données
df = pd.read_csv('data/energy-new.csv', sep=';', na_values=['-', 'ND'], header=0, names=col_names)


In [25]:
# Empty columns
cols_na = df.columns[(df.isna().sum() == df.shape[0])]
df.drop(columns=cols_na, inplace=True)

In [26]:
# Drop also tco anc tco columns since I won't use them
df.drop(columns=['date', 'nature', 'heure', 'tco_thermique', 'tch_thermique', 'tco_nucleaire', 'tch_nucleaire', 'tco_eolien', 'tch_eolien', 'tco_solaire', 'tch_solaire', 'tco_hydraulique', 'tch_hydraulique', 'tco_bioenergies', 'tch_bioenergies'], inplace=True)


In [27]:
# Drop lines if all those columns are Na
cols = ['consommation',
       'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'pompage',
       'bioenergies', 'ech_physiques']

df.dropna(subset=cols, how='all', inplace=True)
df.fillna(0, inplace=True)

In [28]:
# No more Na in the dataset !
df.isna().sum()

code_insee_region    0
region               0
date_heure           0
consommation         0
thermique            0
nucleaire            0
eolien               0
solaire              0
hydraulique          0
pompage              0
bioenergies          0
ech_physiques        0
dtype: int64

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1787316 entries, 12 to 1787327
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   code_insee_region  int64  
 1   region             object 
 2   date_heure         object 
 3   consommation       float64
 4   thermique          float64
 5   nucleaire          float64
 6   eolien             float64
 7   solaire            float64
 8   hydraulique        float64
 9   pompage            float64
 10  bioenergies        float64
 11  ech_physiques      float64
dtypes: float64(9), int64(1), object(2)
memory usage: 177.3+ MB


In [30]:
# Change dtypes to int
cols_int = ['consommation',
       'thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'pompage',
       'bioenergies', 'ech_physiques']

df[cols_int] = df[cols_int].astype(int)

# mW to gW
df[cols_int] = df[cols_int] / 1000

In [31]:
# Sets date_heure to datetime
df['date_heure'] = pd.to_datetime(df['date_heure'], utc=True)
df.dtypes

code_insee_region                  int64
region                            object
date_heure           datetime64[ns, UTC]
consommation                     float64
thermique                        float64
nucleaire                        float64
eolien                           float64
solaire                          float64
hydraulique                      float64
pompage                          float64
bioenergies                      float64
ech_physiques                    float64
dtype: object

In [32]:
# Sorts values
df.sort_values(by='date_heure', ascending=True, inplace=True)
# And resets index
df.reset_index(inplace=True, drop=True)

In [33]:
# Saves in Feather format
df.to_feather('data/energy.ftr')

In [34]:
rs_cols_names = ['date', 'code_insee_region', 'region', 'vitesse_vent', 'rayonnement_solaire']

rs = pd.read_csv('data/rayonnement-solaire-vitesse-vent-new.csv', sep=';', names=rs_cols_names, parse_dates=['date'], header=0)

# Changes dtypes
rs['date'] = pd.to_datetime(rs['date'], utc=True)
rs['vitesse_vent'] = rs['vitesse_vent'].astype(float)
rs['rayonnement_solaire'] = rs['rayonnement_solaire'].astype(float)

# W/m2 to kW/m2
rs['rayonnement_solaire'] = rs['rayonnement_solaire'] / 1000


In [35]:
rs.dtypes


date                   datetime64[ns, UTC]
code_insee_region                    int64
region                              object
vitesse_vent                       float64
rayonnement_solaire                float64
dtype: object

In [36]:
# Sorts values...
rs.sort_values(by='date', ascending=True, inplace=True)
rs.reset_index(inplace=True, drop=True)

In [37]:
# .. and saves in feather format
rs.to_feather('data/rayonnement-new.ftr')

In [38]:
temp_cols_names = ['date', 'code_insee_region', 'region', 'tmin', 'tmax', 'tmoy']
temp = pd.read_csv('data/temperature.csv', sep=';', header=0, names=temp_cols_names)
temp.head()

,date,code_insee_region,region,tmin,tmax,tmoy
0,2019-04-01,32,Hauts-de-France,4.84,17.42,11.13
1,2019-04-02,44,Grand Est,5.34,17.99,11.66
2,2019-04-02,32,Hauts-de-France,7.14,14.48,10.81
3,2019-04-02,52,Pays de la Loire,7.65,13.75,10.70
4,2019-04-02,27,Bourgogne-Franche-Comté,6.18,18.60,12.39


In [39]:
temp.dtypes

date                  object
code_insee_region      int64
region                object
tmin                 float64
tmax                 float64
tmoy                 float64
dtype: object

In [40]:
temp['date'] = pd.to_datetime(temp['date'], utc=True)
temp.dtypes

date                 datetime64[ns, UTC]
code_insee_region                  int64
region                            object
tmin                             float64
tmax                             float64
tmoy                             float64
dtype: object

In [41]:
temp.sort_values(by='date', ascending=True, inplace=True)
temp.reset_index(inplace=True, drop=True)

In [42]:
temp.to_feather('data/temperature.ftr')